# Construct metadata from existing

* Metadata won't have the structure unless it's been filled out first. Starting from scratch means there are certain key-value pairs missing.
* Use 
* Start with a copy of an existing metadata to be template?
* Separate staging XML and overwritten XML, esp now that default template comes with edition = 0...we need to grab that value and increase it by 1

In [1]:
#import xml.etree.ElementTree as ET
#import json
import xmltodict

import metadata_update

In [2]:
XML_FILE = './metadata_xml/ca_hq_transit_areas.xml'
NEW_XML_FILE = './metadata_xml/ca_transit_routes.xml'

hqta = metadata_update.xml_to_json(XML_FILE)
routes = metadata_update.xml_to_json(NEW_XML_FILE)

Loading XML as JSON from ./metadata_xml/ca_hq_transit_areas.xml
Loading XML as JSON from ./metadata_xml/ca_transit_routes.xml


## Lift necessary stuff from 1st time through shp to file gdb

In [3]:
def lift_necessary_dataset_elements(metadata_json):
    m = metadata_json["metadata"]
    
    # Store this info in a dictionary
    d = {}
    
    # Bounding box
    d["idinfo"] = {}
    d["idinfo"]["spdom"] = m["idinfo"]["spdom"] 
    
    # Some description about geospatial layer
    d["spdoinfo"] = m["spdoinfo"] 
    
    # Spatial reference info
    d["spref"] = m["spref"] 
    
    # Field and entity attributes
    d["eainfo"] = m["eainfo"]
    
    return d

In [ ]:
#route_info = lift_necessary_dataset_elements(routes)

In [38]:
def overwrite_default_with_dataset_elements(metadata_json):
    DEFAULT_XML = "./metadata_xml/default.xml"
    default_template = metadata_update.xml_to_json(DEFAULT_XML)
    default = default_template["metadata"]
    
    # Grab the necessary elements from my dataset
    necessary_elements = lift_necessary_dataset_elements(metadata_json)
    
    # Overwrite it in the default template
    for key, value in default.items():
        if key in necessary_elements.keys() and key != "idinfo":
            default[key] = necessary_elements[key]
        
        elif key == "idinfo":
            for k, v in value.items():
                if k == "spdom":
                    default[key][k] = necessary_elements[key][k]
           
            
    # Return the default template, but now with our dataset's info populated
    return default_template

In [39]:
routes_templated = overwrite_default_with_dataset_elements(routes)

Loading XML as JSON from ./metadata_xml/default.xml


In [41]:
def check_edition_add_one(metadata):
    input_edition = (metadata["metadata"]["idinfo"]["citation"]
                     ["citeinfo"]["edition"])
    if input_edition == -999:
        new_edition = 0
    else:
        new_edition = str(int(input_edition) + 1)
    
    return new_edition

In [23]:
# Check if edition is available
# If yes, add 1 to it
# If not, set it to 1

# https://blog.finxter.com/how-to-update-a-key-in-a-dictionary-in-python-if-the-key-doesnt-exist/
#device.setdefault('year',2019)


In [44]:
NEW_EDITION = check_edition_add_one(routes_templated)

SAMPLE_DATASET_INFO = {
    "dataset_name": "my_dataset", 
    "publish_entity": "California Integrated Travel Project", 
    "edition": NEW_EDITION,

    "abstract": "Public. EPSG: 3310",
    "purpose": "Summary sentence about dataset.", 

    "beginning_date": "YYYYMMDD",
    "end_date": "YYYYMMDD",
    "place": "California",

    "status": "Complete", 
    "frequency": "Monthly",
    
    "theme_topics": ["One", "Two", "Three", "Four", "Five"], 

    "methodology": "Detailed methodology description", 
    
    "data_dict_type": "CSV",
    "data_dict_url": "some_url", 

    "contact_organization": "Caltrans", 
    "contact_person": "Analyst Name", 
    "contact_email": "hello@calitp.org" 
}

In [56]:
# Overwrite the metadata after dictionary of dataset info is supplied
def overwrite_metadata_json2(metadata_json, DATASET_INFO):
    d = DATASET_INFO
    new_metadata = metadata_json.copy()
    m = new_metadata["metadata"]
    
    m["idinfo"]["citation"]["citeinfo"]["title"] = d["dataset_name"]
    m["idinfo"]["citation"]["citeinfo"]["pubinfo"]["publish"] = d["publish_entity"]
    ## Need edition and resource contact added to be approved 
    # Add edition 
    # Use number instead of date (shows up when exported in FGDC)
    m["idinfo"]["citation"]["citeinfo"]["edition"] = d["edition"]
    
    m["idinfo"]["descript"]["abstract"] = d["abstract"]
    m["idinfo"]["descript"]["purpose"] = d["purpose"]
    
    m["idinfo"]["timeperd"]["timeinfo"]["rngdates"]["begdate"] = d["beginning_date"]
    m["idinfo"]["timeperd"]["timeinfo"]["rngdates"]["enddate"] = d["end_date"]
    m["idinfo"]["timeperd"]["current"] = d["place"]
    
    m["idinfo"]["status"]["progress"] = d["status"]
    m["idinfo"]["status"]["update"] = d["frequency"]

    m["idinfo"]["keywords"] = d["theme_topics"]    

    # Add resource contact
    m["idinfo"]["ptcontac"]["cntinfo"]["cntorgp"]["cntorg"] = d["contact_organization"]
    m["idinfo"]["ptcontac"]["cntinfo"]["cntorgp"]["cntper"] = d["contact_person"]
    m["idinfo"]["ptcontac"]["cntinfo"]["cntpos"] = d["publish_entity"]
    m["idinfo"]["ptcontac"]["cntinfo"]["cntemail"] = d["contact_email"]    
    
    m["dataqual"]["lineage"]["procstep"]["procdesc"] = d["methodology"]    
    
    m["eainfo"]["detailed"]["enttyp"]["enttypl"] = d["dataset_name"]    
    m["eainfo"]["detailed"]["enttyp"]["enttypd"] = d["data_dict_type"]    
    m["eainfo"]["detailed"]["enttyp"]["enttypds"] = d["data_dict_url"]    
  
    m["metainfo"]["metc"]["cntinfo"]["cntorgp"]["cntorg"] = d["contact_organization"]    
    m["metainfo"]["metc"]["cntinfo"]["cntorgp"]["cntper"] = d["contact_person"]    
    m["metainfo"]["metc"]["cntinfo"]["cntpos"] = d["publish_entity"]    
    m["metainfo"]["metc"]["cntinfo"]["cntemail"] = d["contact_email"]    

    return new_metadata 


In [57]:
routes_templated["metadata"]["eainfo"]["detailed"]["enttyp"]

{'enttypl': 'ca_transit_routes'}

In [58]:
new_metadata = overwrite_metadata_json2(routes_templated, SAMPLE_DATASET_INFO)
